In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import statistics
from scipy import stats
from numba import njit
import pickle
import util as ut

In [ ]:
mask_file="mece_masks_cfos.json"
all_masks=ut.read_json(mask_file)

print(all_masks)
fish_volume_region_FINAL_mini_common=all_masks['mini_masks_common']

In [ ]:
def calculate_vector(fish_volume_region, name_f, treat_name,treatment_or_not, model_organism):
    """
    Compute the vectors across regions and store it as pickle data  
    arguments:
        fish_volume_region: list of regions 
        name_f: folder containing regions tiff stacks 
        treat_name: name of the treatment e.g. shake, food etc
        treatment_or_not: e.g neg
        model_organism: model organism
    returns:
        
    """
    vector_group_neg_red, vector_group_neg_blue = [], []
    with open('%s/%s_%s_distribution_%s_new2_all_mece_MISC_telencephalon.pkl' % (savefolder, treat_name, treatment_or_not, model_organism), 'rb') as f:
        distribution_data = pickle.load(f)

    all_blues = []
    for i, name in enumerate(fish_volume_region):
        filtered_data_fish = ut.filter_data(distribution_data, 'fish', name_f)
        filtered_data = ut.filter_data(filtered_data_fish, 'region', name)
        
        blue_data = ut.filter_data(filtered_data, 'channel', "green")
        blue_dis = blue_data[0]["density_plt"]
        ave_blue = ut.get_number_ave_thres(blue_dis, bottom_threshold, top_threshold)
        all_blues.append(ave_blue)
    
    ave_allblue = np.nanmean(all_blues)
    
    for i, name in enumerate(fish_volume_region):
        
        filtered_data_fish = ut.filter_data(distribution_data, 'fish', name_f)
        filtered_data = ut.filter_data(filtered_data_fish, 'region', name)

        red_data = ut.filter_data(filtered_data, 'channel', "red")
        blue_data = ut.filter_data(filtered_data, 'channel', "green")
        red_dis = red_data[0]["density_plt"]
        blue_dis = blue_data[0]["density_plt"]

        ave_red = ut.get_number_ave_thres(red_dis, bottom_threshold, top_threshold)
        ave_blue = ut.get_number_ave_thres(blue_dis, bottom_threshold, top_threshold)
        vector_group_neg_red.append(ave_red)
        vector_group_neg_blue.append(ave_blue)

    return vector_group_neg_red, vector_group_neg_blue

In [ ]:
def compute_vectors(mask_file, model_organisms, treat_names, fish_names_negs, top_list, 
                    bottom_list, savefolder, treatment_or_not, mask_name):
    """
    Compute the vectors across regions and store it as pickle data  
    arguments:
        mask_file: list of masks
        model_organisms: list of model organisms e.g. ['zebrafish', 'medaka']
        treat_names:name of the treatments ["ms222",  "shake"] 
        fish_names: list of list fish names
        top_list: top intensities [256,256]
        bottom_list: bottom intensities [0,0]
        savefolder: save folder
        treatment_or_not: e.g neg
        mask_name: name of masks e.g. mini_masks_common, macro_masks, mini_masks
    returns:
        
    """
   
    all_masks=ut.read_json(mask_file)
    fish_volume_region=all_masks[mask_name]
    for fish_i, model_organism in enumerate(model_organisms):
        top_threshold=top_list[fish_i]
        bottom_threshold=bottom_list[fish_i]
    for treat_i, treat_name in enumerate(treat_names):
        fishs = fish_names_group[treat_i][fish_i]     
        for name_f_i, name_f in enumerate(fishs):
            vector_group_neg_red, vector_group_neg_blue = calculate_vector(fish_volume_region=fish_volume_region, 
                                                name_f=name_f, treat_name=treat_name, model_organism=model_organism)
            d = {
                "species": model_organism,
                 "treatment":treat_name,
                 "treatment_or_not": treatment_or_not,
                 "vector_red":vector_group_neg_red,
                 "vector_blue":vector_group_neg_blue,
                 "fish_name":name_f
                }    
            vector_groups.append(d)

            
    filename = r"%s/cfos_%s_%s.pkl" % (savefolder, treatment_or_not, mask_name)
    with open(filename, 'wb') as file:
        pickle.dump(vector_groups, file)
    
    

In [ ]:
def plot_manhattans(vector_groups, color):
    """
    Manhattans Plot avg and std using vector group data 
    arguments:
        vector_groups: precomputed vector group
        colors: color assigned to organism
    returns:
        
    """
    means, medians = [], []
    neg_raw_ave, neg_raw_std = [], []
    
    vec_length = len(vector_groups[0])
    for vec_i in range(vec_length):
        vec_i_eachs = []
        for vector_i, vector in enumerate(vector_groups):
            vec_i_eachs.append(vector[vec_i])
        neg_raw_ave.append(np.nanmean(vec_i_eachs))
        neg_raw_std.append(np.nanstd(vec_i_eachs))

    mp_vector_ave = add_element(neg_raw_ave)
    mp_vector_std = add_element(neg_raw_std)
    
    axs.fill_between(xs, np.array(mp_vector_ave) - np.array(mp_vector_std), np.array(mp_vector_ave) + np.array(mp_vector_std),step='pre', color=color, alpha=0.2,)
    axs.step(xs, mp_vector_ave, color=color, alpha=0.8, linewidth=1)
    plt.tight_layout()
    


In [ ]:
def get_vectors(vector_groups, fish_specie,treat_name):
    vector_groups1 = filter_data(vector_groups, "species", fish_specie)
    vector_groups2 = filter_data(vector_groups1, "treatment", treat_name)
        
    vector_C = []
    vector_C,vector_blue = [], []
    for j in range(len(vector_groups2)):
        d = vector_groups2[j]
        vec, vec_blue = [], []
        for k in range(len(d["vector_red"])):
            if d["vector_blue"][k] == 0:
                vec.append(0)
            else:
                vec.append(d["vector_red"][k]/d["vector_blue"][k]) 
                vec_blue.append(d["vector_blue"][k])
        vector_C.append(np.nan_to_num(vec))
        vector_blue.append(np.nan_to_num(vec_blue))
    return  vector_C,vector_blue


def plot_data(mask_file,mask_name,xaxis_mini, fish_species,colors):
    all_masks=ut.read_json(mask_file)
    fish_volume_region=all_masks[mask_name]
    x = np.arange(0, len(fish_volume_region)+1, 1)     
    xs =  x.tolist() + [len(fish_volume_region)]
    plt.axhline(1, linestyle="--", color="black", linewidth=0.5)
    vector_neg_all_m, vector_neg_blue_all_m = get_vectors(vector_groups, fish_specie,treat_name)
    plot_manhattans(vector_groups=vector_neg_all_m, color="orange")
    plot_manhattans(vector_groups=vector_neg_all_z, color="blue")

    for k in xaxis_mini:
        plt.axvline(k, color="grey", alpha=0.1)

    plt.savefig("%s/manhattan_plot_cfos_meg_%s.svg" % (savefolder, mask_name))
    plt.show()


In [ ]:
def test_compute_densities_neg():
    top_list=[256,256]
    bottom_list=[0, 0]
    mask_file="mece_masks_cfos.json"
    mask_name="mini_masks_common"
    treat_names = ["ms222",  "shake"]#"food",
    treatment_or_not = "neg"
    xaxis_mini = [0,3,13,23,26,32,42,52,62,72,82,90,100,106, 107]
    
    fish_names = [[[
                     "fish04",
                     "fish05",
                    ],
                    ["fish02","fish03","fish04",] ],
                  [["fish01",
                    "fish06",
                    "fish07",
                   ],
                   ["fish01","fish02","fish03",]]]
    
    